In [ ]:
%pip install selenium
%pip install beautifulsoup4
%pip install pandas

In [2]:
from selenium import webdriver
from datetime import timedelta, datetime
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.action_chains import ActionChains 
import re

## Сбор данных

In [ ]:
driver = webdriver.Chrome()
driver.get('https://decrypt.co/news-explorer')
#html = driver.page_source
#soup = BeautifulSoup(html)
data = []


while True:
    html = driver.page_source
    soup = BeautifulSoup(html)

    #for a in soup.find_all('a', href=True):
        #if len(re.findall(r"(\d+\/)(\S*)",a['href'])) or len(re.findall(r"(resources\/)(\S*)",a['href'])):
            #print("Found the URL:", a['href'])
            #pass
    
    containers = soup.find_all("div",attrs={"class":"flex gap-2 flex-col py-5 px-4 pb-3 md:p-6 md:pb-4 border-[1px] border-solid border-b-0 undefined relative"})
    for i, d in enumerate(containers):
        header = d.find("h4", {"class": "font-akzidenz-grotesk font-medium text-base md:text-xl degen-alley-dark:text-white"})
        if header is None:
            header = "NOT FOUND"
        else:
            header = header.text.strip()

        timedata = d.find("div", {"class": "text-sm text-neutral-700 degen-alley-dark:text-neutral-200"})
        if timedata is None:
            timedata = "NOT FOUND"
        else:
            timedata = timedata.text.strip()

        ref = d.find("a", {"class": "py-2 text-center space-x-2 font-akzidenz-grotesk font-medium text-sm leading-none"})
        if ref is None:
            ref = "NOT FOUND"
        else:
            ref = ref['href'].strip()

        #if (header, timedata, ref) in data:
        #    continue
        if (header, timedata, ref) in data:
            continue
        
        data.append((header, timedata, ref))
        print(f"Found header: {header}, date: {timedata}, ref: {ref}")
        #time_data = soup.find_all("div",attrs={"class":"post-loop__date"})   
    #print(len(data))
    button = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[1]/div/main/div[4]/div/div[1]/div/button')
    location = button.location
    size = button.size
    w, h = size['width'], size['height']
    driver.execute_script(f"window.scrollTo(0, {h});")
    driver.execute_script("arguments[0].click();", button)
    time.sleep(1)


## Обработка данных

In [ ]:
df = pd.DataFrame(data, columns = ['header', 'date', 'link'])
df = df.drop_duplicates()
df.head()

In [11]:
df.to_csv("../data/decrypt_no_index.csv", index=False)

In [10]:
df.tail()
#df = df.drop(columns='Unnamed: 0')
df = df.rename(columns={"href": "link"})
df.tail()

,header,date,link
7821,Eclipse Launches Software to Bridge Solana and...,"Feb 23, 11:00 pm",https://techcrunch.com/2023/02/23/eclipse-laun...
7822,Dapper Labs Further Cuts Staff by 20% Despite ...,"Feb 23, 11:58 pm",https://www.coindesk.com/business/2023/02/23/d...
7823,Optimism Pumps 16% Amid Coinbase Layer-2 Launch,"Feb 23, 7:12 pm",https://decrypt.co/122029/optimism-pumps-coinb...
7824,DeFi Protocol Platypus Works With Binance to I...,"Feb 23, 9:04 pm",https://www.coindesk.com/tech/2023/02/23/defi-...
7825,Government's New Indictment Increases Focus on...,"Feb 23, 8:32 pm",https://www.axios.com/2023/02/23/ftx-case-sbf-...


## Чтение данных из файла

In [29]:
df = pd.read_csv("../data/decrypt_no_index.csv")

# Маркировка по годам

In [30]:
print(list(df['date']))

['Apr 9, 9:12 am', 'Apr 9, 8:08 am', 'Apr 9, 6:16 am', 'Apr 9, 4:13 am', 'Apr 9, 4:04 am', 'Apr 9, 3:17 am', 'Apr 9, 2:18 am', 'Apr 9, 1:49 am', 'Apr 9, 12:53 am', 'Apr 8, 11:56 pm', 'Apr 8, 10:20 pm', 'Apr 8, 9:09 pm', 'Apr 8, 7:53 pm', 'Apr 8, 7:28 pm', 'Apr 8, 7:06 pm', 'Apr 8, 6:46 pm', 'Apr 8, 6:11 pm', 'Apr 8, 5:54 pm', 'Apr 8, 5:53 pm', 'Apr 8, 4:06 pm', 'Apr 8, 4:00 pm', 'Apr 8, 3:01 pm', 'Apr 8, 4:12 pm', 'Apr 8, 9:15 am', 'Apr 8, 8:12 am', 'Apr 8, 6:05 am', 'Apr 8, 5:19 am', 'Apr 8, 2:41 am', 'Apr 8, 12:50 am', 'Apr 8, 12:07 am', 'Apr 7, 11:20 pm', 'Apr 7, 9:47 pm', 'Apr 7, 9:19 pm', 'Apr 7, 7:15 pm', 'Apr 7, 6:54 pm', 'Apr 7, 6:18 pm', 'Apr 7, 4:40 pm', 'Apr 7, 4:35 pm', 'Apr 7, 2:47 pm', 'Apr 7, 1:43 pm', 'Apr 7, 12:38 pm', 'Apr 7, 9:54 am', 'Apr 7, 9:57 am', 'Apr 7, 8:28 am', 'Apr 7, 7:26 am', 'Apr 7, 6:24 am', 'Apr 7, 5:16 am', 'Apr 7, 3:03 am', 'Apr 5, 1:35 am', 'Apr 5, 12:02 am', 'Apr 4, 11:06 pm', 'Apr 4, 10:00 pm', 'Apr 4, 9:50 pm', 'Apr 4, 9:51 pm', 'Apr 4, 8:53 pm',

In [31]:
current_year = 2025
reached_30dec = False

for i, dtime in enumerate(list(df['date'])):
    year_counted = current_year
    cdate, ctime = [s.strip() for s in dtime.split(',')]
    #print(cdate, ctime)
    
    if cdate == "Dec 30":
        reached_30dec = True
        year_counted -= 1
    elif reached_30dec:
        current_year -= 1
        reached_30dec = False
    
    df.loc[i, 'date'] = f"{cdate}, {ctime}, {year_counted}"
    
df

,header,date,link
0,Bitcoin and Ethereum Prices Plummet Due to Whi...,"Apr 9, 9:12 am, 2025",https://decrypt.co/314059/bitcoin-ethereum-dip...
1,New York Eyes Blockchain's Role in Safeguardin...,"Apr 9, 8:08 am, 2025",https://decrypt.co/314057/new-york-bill-blockc...
2,Hawk Tuah Token Controversy Notably Absent in ...,"Apr 9, 6:16 am, 2025",https://decrypt.co/314049/hawk-tuah-podcast-ks...
3,Argentine Chamber of Deputies Votes to Investi...,"Apr 9, 4:13 am, 2025",https://decrypt.co/314041/argentine-congress-a...
4,Winklevoss-Backed AI Startup Raises Millions t...,"Apr 9, 4:04 am, 2025",https://decrypt.co/314038/winklevoss-backed-st...
...,...,...,...
7821,Eclipse Launches Software to Bridge Solana and...,"Feb 23, 11:00 pm, 2023",https://techcrunch.com/2023/02/23/eclipse-laun...
7822,Dapper Labs Further Cuts Staff by 20% Despite ...,"Feb 23, 11:58 pm, 2023",https://www.coindesk.com/business/2023/02/23/d...
7823,Optimism Pumps 16% Amid Coinbase Layer-2 Launch,"Feb 23, 7:12 pm, 2023",https://decrypt.co/122029/optimism-pumps-coinb...
7824,DeFi Protocol Platypus Works With Binance to I...,"Feb 23, 9:04 pm, 2023",https://www.coindesk.com/tech/2023/02/23/defi-...


In [32]:
df.to_csv("../data/decrypt_with_years.csv", index=False)